### Coding Exercises

Consider the following classes:

In [1]:
class Stock:
    def __init__(self, symbol, date, open_, high, low, close, volume):
        self.symbol = symbol
        self.date = date
        self.open = open_
        self.high = high
        self.low = low
        self.close = close
        self.volume = volume
        
class Trade:
    def __init__(self, symbol, timestamp, order, price, volume, commission):
        self.symbol = symbol
        self.timestamp = timestamp
        self.order = order
        self.price = price
        self.commission = commission
        self.volume = volume

#### Exercise 1

Given the above class, write a custom `JSONEncoder` class to **serialize** dictionaries that contain instances of these particular classes. Keep in mind that you will want to deserialize the data too - so you will need some technique to indicate the object type in your serialization.

For example you may have an object such as this one that needs to be serialized:

In [2]:
from datetime import date, datetime
from decimal import Decimal

activity = {
    "quotes": [
        Stock('TSLA', date(2018, 11, 22), 
              Decimal('338.19'), Decimal('338.64'), Decimal('337.60'), Decimal('338.19'), 365_607),
        Stock('AAPL', date(2018, 11, 22), 
              Decimal('176.66'), Decimal('177.25'), Decimal('176.64'), Decimal('176.78'), 3_699_184),
        Stock('MSFT', date(2018, 11, 22), 
              Decimal('103.25'), Decimal('103.48'), Decimal('103.07'), Decimal('103.11'), 4_493_689)
    ],
    
    "trades": [
        Trade('TSLA', datetime(2018, 11, 22, 10, 5, 12), 'buy', Decimal('338.25'), 100, Decimal('9.99')),
        Trade('AAPL', datetime(2018, 11, 22, 10, 30, 5), 'sell', Decimal('177.01'), 20, Decimal('9.99'))
    ]
}

Hint: You can modify the classes if you need to.

In [3]:
import json

In [4]:
class CustomEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, Stock):
            return {'_type': 'Stock',
                    'symbol':obj.symbol, 
                    'date':obj.date,
                    'open':obj.open,
                    'high':obj.high,
                    'low':obj.low,
                    'close':obj.close,
                    'volume':obj.volume}
        if isinstance(obj, Trade):
            return {'_type': 'Trade',
                    'symbol':obj.symbol, 
                    'timestamp':obj.timestamp,
                    'order':obj.order,
                    'price':obj.price,
                    'volume':obj.volume,
                    'commission':obj.commission}
        if isinstance(obj, date):
            return obj.isoformat()
        if isinstance(obj, datetime):
            return obj.isoformat()
        if isinstance(obj, Decimal):
            return obj.to_eng_string()
        return super().default(obj)

In [5]:
encoded = json.dumps(activity, cls=CustomEncoder, indent=2)
print(encoded)

{
  "quotes": [
    {
      "_type": "Stock",
      "symbol": "TSLA",
      "date": "2018-11-22",
      "open": "338.19",
      "high": "338.64",
      "low": "337.60",
      "close": "338.19",
      "volume": 365607
    },
    {
      "_type": "Stock",
      "symbol": "AAPL",
      "date": "2018-11-22",
      "open": "176.66",
      "high": "177.25",
      "low": "176.64",
      "close": "176.78",
      "volume": 3699184
    },
    {
      "_type": "Stock",
      "symbol": "MSFT",
      "date": "2018-11-22",
      "open": "103.25",
      "high": "103.48",
      "low": "103.07",
      "close": "103.11",
      "volume": 4493689
    }
  ],
  "trades": [
    {
      "_type": "Trade",
      "symbol": "TSLA",
      "timestamp": "2018-11-22T10:05:12",
      "order": "buy",
      "price": "338.25",
      "volume": 100,
      "commission": "9.99"
    },
    {
      "_type": "Trade",
      "symbol": "AAPL",
      "timestamp": "2018-11-22T10:30:05",
      "order": "sell",
      "price": "177.01"

#### Exercise 2

Write code to reverse the serialization you just created. Write a custom decoder that can deserialize a JSON structure containing `Stock` and `Trade` objects. 

In [6]:
class CustomDecoder(json.JSONDecoder):
    
    def decode(self, arg):
        obj = json.loads(arg)
        return self.parse_obj(obj)
        
    def parse_obj(self, obj):
        if isinstance(obj, dict):
            if '_type' in obj:
                if obj['_type'] == 'Stock':
                    return Stock(obj['symbol'], 
                                 datetime.fromisoformat(obj['date']).date(), 
                                 Decimal(obj['open']), 
                                 Decimal(obj['high']), 
                                 Decimal(obj['low']), 
                                 Decimal(obj['close']), 
                                 obj['volume'])
                if obj['_type'] == 'Trade':
                    return Trade(obj['symbol'], 
                                 datetime.fromisoformat(obj['timestamp']), 
                                 obj['order'], 
                                 Decimal(obj['price']), 
                                 obj['volume'],
                                 Decimal(obj['commission']))
            for k, v in obj.items():
                obj[k] = self.parse_obj(v)
        
        if isinstance(obj, list):
            obj = [*map(self.parse_obj, obj)]
        
        return obj

In [7]:
decoded = json.loads(encoded, cls=CustomDecoder)
decoded

{'quotes': [<__main__.Stock at 0x7f823504f970>,
 'trades': [<__main__.Trade at 0x7f823504f9d0>,
  <__main__.Trade at 0x7f823504fcd0>]}

In [8]:
for v in decoded['quotes']:
    print(v.symbol, type(v.symbol), 
          v.date, type(v.date), 
          v.open, type(v.open), 
          v.high, type(v.high), 
          v.low, type(v.low), 
          v.close, type(v.close), 
          v.volume, type(v.volume))
for v in decoded['trades']:
    print(v.symbol, type(v.symbol), 
          v.timestamp, type(v.timestamp), 
          v.order, type(v.order), 
          v.price, type(v.price),
          v.volume, type(v.volume),
          v.commission, type(v.commission))

TSLA <class 'str'> 2018-11-22 <class 'datetime.date'> 338.19 <class 'decimal.Decimal'> 338.64 <class 'decimal.Decimal'> 337.60 <class 'decimal.Decimal'> 338.19 <class 'decimal.Decimal'> 365607 <class 'int'>
AAPL <class 'str'> 2018-11-22 <class 'datetime.date'> 176.66 <class 'decimal.Decimal'> 177.25 <class 'decimal.Decimal'> 176.64 <class 'decimal.Decimal'> 176.78 <class 'decimal.Decimal'> 3699184 <class 'int'>
MSFT <class 'str'> 2018-11-22 <class 'datetime.date'> 103.25 <class 'decimal.Decimal'> 103.48 <class 'decimal.Decimal'> 103.07 <class 'decimal.Decimal'> 103.11 <class 'decimal.Decimal'> 4493689 <class 'int'>
TSLA <class 'str'> 2018-11-22 10:05:12 <class 'datetime.datetime'> buy <class 'str'> 338.25 <class 'decimal.Decimal'> 100 <class 'int'> 9.99 <class 'decimal.Decimal'>
AAPL <class 'str'> 2018-11-22 10:30:05 <class 'datetime.datetime'> sell <class 'str'> 177.01 <class 'decimal.Decimal'> 20 <class 'int'> 9.99 <class 'decimal.Decimal'>


#### Exercise 3

Do the same serialization and deserialization, but using `Marshmallow`.